In [1]:
import subprocess, cv2, json, os, sys, shutil, pyk4a, time
import numpy as np, matplotlib.pyplot as plt
from video_io import *

ModuleNotFoundError: No module named 'cv2'

In [2]:
filename_prefix = 'data/22_2_2_GRIN61_MeA8A_training'
recording_length = 5400

filename_prefix += '.rig2'

In [3]:
if os.path.exists(filename_prefix+'.bottom.ir.avi'):
    print('A RECORDING WITH THIS NAME EXISTS')
else:
    start_recording(filename_prefix, recording_length,top_device_id=0, bottom_device_id=1, display='bottom')

Recorded 4890 out of 5400 seconds

Process Process-1:1:
Process Process-2:1:
Process Process-2:2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dattalab2/miniconda3/envs/k4a/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/dattalab2/miniconda3/envs/k4a/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/dattalab2/miniconda3/envs/k4a/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dattalab2/miniconda3/envs/k4a/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dattalab2/Desktop/22_1_13_realtime_tracking_for_untangling/video_io.py", line 211, in display_images
    data = display_queue.get()
  File "/home/dattalab2/Desktop/22_1_13_realtime_tracking_for_untangling/video_io.py", line 227, in write_images
    data = image_queue.

 - Frame rate =  29.98886452444962
 - Frame rate =  29.98922609677796
